In [1]:
import json
import os

import argparse
import torch
import yaml
import shutil
from helper import *
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from efficientdet_arch.backbone import EfficientDetBackbone
from efficientdet_arch.efficientdet.utils import BBoxTransform, ClipBoxes
from efficientdet_arch.utils.utils import preprocess, invert_affine, postprocess, boolean_string

In [2]:
INPUT_DIM = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]

PROJECT_NAME = "goodbadchili"
CLASSES = ["good_chili", "bad_chili"]
EFFICIENTNET_COMPOUND_COEF = 0
WEIGHT_PATH = "d0-chili-100E.pth"
ANCHOR_RATIOS=[(1.0, 0.7), (1.0, 1.0), (1.0, 1.5)]
ANCHOR_SCALES=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

THRESHOLD=0.3
NMS_THRESHOLD=0.5

In [3]:
PROJECT_DIR = os.path.join(os.path.abspath(os.getcwd()), "datasets", PROJECT_NAME, "COCO")
RESULT_FNAME = f"{WEIGHT_PATH[:-4]}_results.json"
ANNOTATION_PATH = os.path.join(PROJECT_DIR, "annotations", "instances_test.json")
IMAGE_DIR = os.path.join(PROJECT_DIR, "test")
COCO_INFO = COCO(ANNOTATION_PATH)
IMAGE_IDS = COCO_INFO.getImgIds()[:10000]
RESULT_IMG_DIR = os.path.join(os.path.abspath(os.getcwd()), "results", WEIGHT_PATH[:-4])

try:
    os.mkdir(RESULT_IMG_DIR)
except FileExistsError:
    shutil.rmtree(RESULT_IMG_DIR)
    os.mkdir(RESULT_IMG_DIR)
    pass

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
model = EfficientDetBackbone(num_classes=len(CLASSES), compound_coef=EFFICIENTNET_COMPOUND_COEF, ratios=ANCHOR_RATIOS, scales=ANCHOR_SCALES)
model.load_state_dict(torch.load(os.path.join("weights", WEIGHT_PATH), map_location=torch.device('cpu')))
model.requires_grad_(False)
model.eval()
model.cuda("cuda:0")

EfficientDetBackbone(
  (bifpn): Sequential(
    (0): BiFPN(
      (conv6_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv5_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv4_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
      

In [5]:
results = []
regressBoxes, clipBoxes = BBoxTransform(), ClipBoxes()
for image_id in tqdm(IMAGE_IDS):
    image_info = COCO_INFO.loadImgs(image_id)[0]
    image_path = os.path.join(IMAGE_DIR, image_info['file_name'])
    og, framed_imgs, framed_metas = preprocess(image_path, max_size=INPUT_DIM[EFFICIENTNET_COMPOUND_COEF], mean=CONFIG["mean"], std=CONFIG["std"])
    x = torch.from_numpy(framed_imgs[0]).cuda("cuda:0").float().unsqueeze(0).permute(0, 3, 1, 2)
    _, regression, classification, anchors = model(x)
    preds = postprocess(x, anchors, regression, classification, regressBoxes, clipBoxes, THRESHOLD, NMS_THRESHOLD)
    if not preds: continue
    preds = invert_affine(framed_metas, preds)[0]
    scores, class_ids, rois = preds['scores'], preds['class_ids'], preds['rois']
    if rois.shape[0] > 0:   # x1,y1,x2,y2 -> x1,y1,w,h
        rois[:, 2] -= rois[:, 0]
        rois[:, 3] -= rois[:, 1]
        bbox_score = scores
        for roi_id in range(rois.shape[0]):
            score, label, box = float(bbox_score[roi_id]), int(class_ids[roi_id]), rois[roi_id, :]
            image_result = {
                'image_id': image_id,
                'category_id': label + 1,
                'score': float(score),
                'bbox': box.tolist(),
            }
            results.append(image_result)
            (x1, y1, w, h) = box.astype(np.int_)
            cls = CLASSES[label]
            color = (0, 255, 0) if label == 0 else (0, 0, 255)      # TODO: change different dataset
            cv2.rectangle(og[0], (x1, y1), (x1+w, y1+h), color, 2)
            cv2.putText(og[0], f"{cls}, {score:.3f}", (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        cv2.imwrite(os.path.join(RESULT_IMG_DIR, f"{image_id}.jpg"), og[0])
        
if len(results) == 0:
    raise Exception('Model does not produce any valid output, check model input')

with open(os.path.join(os.path.abspath(os.getcwd()), "results", RESULT_FNAME), "w") as f:
    json.dump(results, f, indent=4)

100%|████████████████████████████████████████████████████████████████████████████████| 849/849 [01:39<00:00,  8.57it/s]


In [6]:
coco_eval = COCOeval(COCO_INFO, COCO_INFO.loadRes(os.path.join(os.path.abspath(os.getcwd()), "results", RESULT_FNAME)), 'bbox')
coco_eval.params.imgIds = IMAGE_IDS
coco_eval.evaluate()
coco_eval.accumulate()

Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.12s).


In [7]:
coco_eval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.216
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.324
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.390
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.406
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.378
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.557
